In [1]:
# Installer ipywidgets si ce n'est pas déjà fait
# !pip install ipywidgets

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from collections import defaultdict
import itertools

# ------------------ CONFIGURATION ------------------

unit_list = [
    "", "Archer", "Paladin", "Soldat", "Mage", "Dragon", "Erudit",
    "Garde Royale", "Ranger", "Nécromancien", "Porte Étendard",
    "Cartographe", "Guérisseur", "Golem", "Ménestrel", "Assassin"
]

# Force de base pour chaque unité
base_force = {
    "Archer": 2, "Paladin": 4, "Soldat": 3, "Mage": 3, "Dragon": 10,
    "Erudit": 0, "Garde Royale": 5, "Ranger": 3, "Nécromancien": 6,
    "Porte Étendard": 0, "Cartographe": 6, "Guérisseur": 3,
    "Golem": 5, "Ménestrel": 0, "Assassin": 4
}

# Mappages pour les bonus de suite
suite_bonus_joueur = {2: 1, 3: 4, 4: 7, 5: 12}
suite_bonus_ia = {2: 3, 3: 7, 4: 12, 5: 20}

grid_size = 3

# ------------------ OUTILS ------------------
def get_adjacent(x, y):
    """Retourne la liste des coordonnées adjacentes (orthogonales) d'une case dans la grille 3×3."""
    return [(x+dx, y+dy) for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)] if 0 <= x+dx < grid_size and 0 <= y+dy < grid_size]

def count_units(grid):
    """Compte le nombre d'occurrences de chaque unité dans une grille."""
    counter = defaultdict(int)
    for row in grid:
        for unit, _ in row:
            if unit:
                counter[unit] += 1
    return counter

def get_base_force(grid):
    """Calcule la force de base totale d'une grille."""
    total = 0
    for row in grid:
        for unit, _ in row:
            if unit in base_force:
                total += base_force[unit]
    return total

def extract_values(grid):
    """Extrait la grille des valeurs de position à partir d'une grille de tuples (unit, value)."""
    return [[val for _, val in row] for row in grid]

def get_suite_bonus(grid_values, player_type):
    """
    Parcourt la grille de valeurs de position (3×3) et détecte
    les régions connexes (adjacentes orthogonalement) dont les valeurs forment un ensemble consécutif.
    Chaque région ne peut être utilisée qu'une seule fois. Les points attribués sont
    selon le mapping (pour le Joueur ou l'IA).
    """
    visited = [[False]*grid_size for _ in range(grid_size)]
    suites = []

    def dfs(x, y, region):
        visited[x][y] = True
        region.append((x, y))
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            nx, ny = x+dx, y+dy
            if 0 <= nx < grid_size and 0 <= ny < grid_size and not visited[nx][ny]:
                region.append((nx, ny))
                # On ne regarde pas la relation entre chaque case,
                # On formera la région complète puis on vérifiera la consécutivité
                dfs(nx, ny, region)

    all_coords = [(i, j) for i in range(grid_size) for j in range(grid_size)]
    used = set()
    total_bonus = 0
    for (x, y) in all_coords:
        if (x, y) not in used:
            region = []
            dfs(x, y, region)
            # Trier les valeurs dans la région
            vals = [grid_values[i][j] for i,j in region]
            mini, maxi = min(vals), max(vals)
            # Vérifie si la région forme une suite : max - min = nombre de cases - 1
            if maxi - mini == len(region) - 1 and len(region) >= 2:
                # Attribuer les points selon la longueur (en se bornant à 5 si plus de 5 cases)
                l = len(region)
                if l > 5:
                    l = 5
                if player_type == "Joueur":
                    total_bonus += suite_bonus_joueur.get(l, 0)
                else:
                    total_bonus += suite_bonus_ia.get(l, 0)
                used.update(region)
    return total_bonus

# ------------------ BONUS UNITÉ + SUITE ------------------
def get_bonus(grid, player_type, suite_bonus):
    bonus = suite_bonus
    unit_counts = count_units(grid)

    for x in range(grid_size):
        for y in range(grid_size):
            unit, val = grid[x][y]
            if not unit:
                continue
            # Récupère la liste des unités adjacentes (seulement le nom)
            adj_units = [grid[i][j][0] for i, j in get_adjacent(x, y) if grid[i][j][0]]

            if unit == "Archer":
                if player_type == "Joueur":
                    bonus += 4 * adj_units.count("Archer")
                else:
                    bonus += 4 * (unit_counts["Archer"] - 1)

            elif unit == "Paladin":
                if player_type == "Joueur":
                    bonus += 3 * adj_units.count("Paladin")
                else:
                    bonus += 3 * (unit_counts["Paladin"] - 1)

            elif unit == "Soldat":
                if player_type == "Joueur":
                    bonus += 3 * (unit_counts["Soldat"] - 1)
                else:
                    bonus += 4 * (unit_counts["Soldat"] - 1)

            elif unit == "Mage":
                if player_type == "Joueur":
                    bonus += 2 * adj_units.count("Mage")
                else:
                    bonus += 2 * (unit_counts["Mage"] - 1)

            elif unit == "Dragon":
                if player_type == "Joueur":
                    if any(u in ["Mage", "Soldat", "Archer", "Paladin"] for u in adj_units):
                        bonus -= 5
                else:
                    if any(u in ["Mage", "Archer"] for u in adj_units):
                        bonus -= 4

            elif unit == "Erudit":
                types = ["Mage", "Guérisseur", "Nécromancien"]
                bonus += (2 if player_type == "Joueur" else 3) * sum(unit_counts[t] for t in types)

            elif unit == "Garde Royale":
                nb_soldats = unit_counts["Soldat"]
                nb_paladins = unit_counts["Paladin"]
                bonus += (2 if player_type == "Joueur" else 3) * max(nb_soldats, nb_paladins)

            elif unit == "Ranger":
                bonus += (2 if player_type == "Joueur" else 3) * (unit_counts["Mage"] + unit_counts["Archer"])

            elif unit == "Guérisseur":
                for u in adj_units:
                    if u in base_force and base_force[u] <= 3:
                        bonus += 3 if player_type == "Joueur" else 4

            elif unit == "Golem":
                base_units = ["Archer", "Paladin", "Soldat", "Mage"]
                adj_count = sum(1 for u in adj_units if u in base_units)
                if player_type == "Joueur":
                    if adj_count == 2:
                        bonus += 5
                    elif adj_count == 0:
                        bonus -= 3
                else:
                    if adj_count >= 1:
                        bonus += 5

            elif unit == "Ménestrel":
                if player_type == "Joueur":
                    bonus += len(set(u for row in grid for u, _ in row if u))
                else:
                    ligne = [grid[x][j][0] for j in range(3)]
                    colonne = [grid[i][y][0] for i in range(3)]
                    bonus += 2 * len(set(ligne + colonne))

            elif unit == "Assassin":
                odd_count = sum(1 for u in adj_units if u in base_force and base_force[u] % 2 == 1)
                if player_type == "Joueur":
                    bonus += 4 if odd_count == 2 else 9 if odd_count >= 3 else 0
                else:
                    bonus += 4 if odd_count == 1 else 9 if odd_count >= 2 else 0

            elif unit == "Nécromancien":
                bonus += (3 if player_type == "Joueur" else 4) * suite_bonus

            elif unit == "Porte Étendard":
                bonus += (2 if player_type == "Joueur" else 3) * (suite_bonus + 1 if suite_bonus > 0 else 0)
    return bonus

# ------------------ INTERFACE ------------------

# Crée deux grilles interactives : une pour le Joueur et une pour l'IA.
# Pour chaque case, on choisit l'unité et la valeur de position.
unit_widgets = []   # Pour la sélection d'unité
value_widgets = []  # Pour la saisie de la valeur de position

for label in ["Joueur", "IA"]:
    display(HTML(f"<h3>{label}</h3>"))
    row_units = []
    row_values = []
    for i in range(3):
        unit_row = []
        value_row = []
        for j in range(3):
            unit_dd = widgets.Dropdown(options=unit_list, layout=widgets.Layout(width="150px"))
            val_dd = widgets.Dropdown(options=list(range(1, 9)), layout=widgets.Layout(width="60px"))
            box = widgets.VBox([unit_dd, val_dd])
            display(box)
            unit_row.append(unit_dd)
            value_row.append(val_dd)
        row_units.append(unit_row)
        row_values.append(value_row)
    unit_widgets.append(row_units)
    value_widgets.append(row_values)

# ------------------ FONCTIONS DE CALCUL ------------------

def calculate(_):
    clear_output(wait=True)
    # Reconstruction des grilles pour Joueur et IA, chaque cellule = (unité, valeur)
    joueur_grid = [[(unit_widgets[0][i][j].value, value_widgets[0][i][j].value) for j in range(3)] for i in range(3)]
    ia_grid = [[(unit_widgets[1][i][j].value, value_widgets[1][i][j].value) for j in range(3)] for i in range(3)]

    # Extraction des valeurs de position
    joueur_values = extract_values(joueur_grid)
    ia_values = extract_values(ia_grid)

    # Calcul des bonus suite à partir des valeurs de position
    bonus_suite_joueur = get_suite_bonus(joueur_values, "Joueur")
    bonus_suite_ia = get_suite_bonus(ia_values, "IA")

    # Calcul de la force de base
    base_j = get_base_force(joueur_grid)
    base_ia = get_base_force(ia_grid)

    # Calcul total des bonus (inclus bonus suite)
    bonus_j = get_bonus(joueur_grid, "Joueur", bonus_suite_joueur)
    bonus_ia = get_bonus(ia_grid, "IA", bonus_suite_ia)

    total_j = base_j + bonus_j
    total_ia = base_ia + bonus_ia

    print("Grille Joueur (Unité, Valeur) :")
    for row in joueur_grid:
        print(row)
    print(f"\nForce de base Joueur : {base_j}")
    print(f"Bonus Joueur (incluant suites) : {bonus_j}")
    print(f"Score total Joueur : {total_j}")

    print("\nGrille IA (Unité, Valeur) :")
    for row in ia_grid:
        print(row)
    print(f"\nForce de base IA : {base_ia}")
    print(f"Bonus IA (incluant suites) : {bonus_ia}")
    print(f"Score total IA : {total_ia}")

def get_suite_bonus(grid_values, player_type):
    """
    Parcourt la grille de valeurs de position et détecte les suites connexes.
    Une suite se définit si l'ensemble des valeurs dans une région connexe (adjacente orthogonalement)
    forme une séquence consécutive (peu importe l'ordre).
    Chaque suite rapporte des points selon le mapping suivant :
       Joueur : 2→1, 3→4, 4→7, 5 ou plus →12
       IA     : 2→3, 3→7, 4→12, 5 ou plus →20
    Une case ne peut participer qu'à une seule suite, et on priorise les suites les plus longues.
    """
    visited = [[False]*3 for _ in range(3)]
    suites = []

    def dfs(x, y, region):
        visited[x][y] = True
        region.append((x, y))
        for dx, dy in [(-1,0),(1,0),(0,-1),(0,1)]:
            nx, ny = x+dx, y+dy
            if 0 <= nx < 3 and 0 <= ny < 3 and not visited[nx][ny]:
                dfs(nx, ny, region)

    # Détecte les régions connexes (pas encore filtrées par suite consécutive)
    for i in range(3):
        for j in range(3):
            if not visited[i][j]:
                region = []
                dfs(i, j, region)
                if len(region) >= 2:
                    # Vérifie si les valeurs de la région forment une suite consécutive
                    values = [grid_values[x][y] for x, y in region]
                    mini, maxi = min(values), max(values)
                    if maxi - mini == len(region) - 1:
                        suites.append(region)

    # Attribue des points par suite selon le mapping
    bonus = 0
    for region in suites:
        l = len(region)
        if player_type == "Joueur":
            if l == 2:
                bonus += 1
            elif l == 3:
                bonus += 4
            elif l == 4:
                bonus += 7
            elif l >= 5:
                bonus += 12
        else:  # IA
            if l == 2:
                bonus += 3
            elif l == 3:
                bonus += 7
            elif l == 4:
                bonus += 12
            elif l >= 5:
                bonus += 20
    return bonus

btn = widgets.Button(description="Calculer les scores", button_style="success")
btn.on_click(calculate)
display(btn)


Button(button_style='success', description='Calculer les scores', style=ButtonStyle())